In [3]:
import numpy as np
import arepo
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from tqdm import tqdm
import matplotlib as mpl
from scipy.stats import binned_statistic_2d

In [4]:
basepath = '/n/holylfs05/LABS/hernquist_lab/Users/abeane/GSEgas/'

In [34]:
sn = arepo.Snapshot(basepath+'ics/MW3iso_fg0.7_MHG0.15_RC9/lvl4/MW.hdf5')

In [35]:
sn.part0.GFM_Metallicity

array([0.5011872 , 0.5011872 , 0.5011872 , ..., 0.05011872, 0.05011872,
       0.05011872], dtype=float32)

In [36]:
sn.part0.GFM_Metals[0]

array([7.4937481e-01, 2.4426009e-01, 1.2028493e-03, 3.5083108e-04,
       2.8567673e-03, 6.0142466e-04, 3.5083108e-04, 3.5083108e-04,
       6.5154338e-04, 0.0000000e+00], dtype=float32)

In [25]:
sn = arepo.Snapshot(basepath+'runs/MW3iso_fg0.7_MHG0.15_RC9/lvl4/output', 0, combineFiles=True)

In [29]:
np.where(sn.part0.GFM_Metallicity > 0.0006366)

(array([   218,    219,    220, ..., 458266, 458267, 458268]),)

In [30]:
sn.part0.GFM_Metallicity[218]

0.006365078

In [32]:
0.5011872 * 0.0127

0.00636507744

In [33]:
sn.part0.GFM_Metals[0]

array([7.5893748e-01, 2.4042600e-01, 1.2028494e-04, 3.5083107e-05,
       2.8567671e-04, 6.0142469e-05, 3.5083107e-05, 3.5083107e-05,
       6.5154338e-05, 0.0000000e+00], dtype=float32)

In [37]:
sn = arepo.Snapshot(basepath+'runs/MW3iso_fg0.7_MHG0.15_RC9-nometals/lvl4/output', 0, combineFiles=True)

In [38]:
sn.part0.GFM_Metallicity

array([0.0127, 0.0127, 0.0127, ..., 0.0127, 0.0127, 0.0127], dtype=float32)

## baryon fraction

In [25]:
R200_MW = 129
sn_MW = arepo.Snapshot(basepath+'ics/MW3iso_fg0.7_MHG0.25_RC9/lvl4/MW.hdf5')

In [24]:
R200_GSE = 97.5
sn_GSE = arepo.Snapshot(basepath+'ics/GSE2iso_fg0.7_MHG0.18_RC6.5/lvl4/GSE.hdf5')

In [28]:
def print_bary_dm_frac(sn, R200):
    center = np.array([200, 200, 200])
    rpart0 = np.linalg.norm(sn.part0.pos - center, axis=1)
    rpart1 = np.linalg.norm(sn.part1.pos - center, axis=1)
    rpart2 = np.linalg.norm(sn.part2.pos - center, axis=1)
    
    key0 = rpart0 < R200
    key1 = rpart1 < R200
    key2 = rpart2 < R200
    
    mass0 = np.sum(sn.part0.mass[key0])
    mass1 = sn.MassTable[1] * len(np.where(key1)[0])
    mass2 = sn.MassTable[2] * len(np.where(key2)[0])
    
    mass3 = 0
    if sn.NumPart_Total[3] > 0:
        rpart3 = np.linalg.norm(sn.part3.pos - center, axis=1)
        key3 = rpart3 < R200_MW
        mass3 = sn.MassTable[3] * len(np.where(key3)[0])
        
    mass_bary = mass0 + mass2 + mass3
    mass_dm = mass1

    print('bary/dm=', mass_bary/mass_dm)
    print('planck=', 0.022068/0.12029)
    print('fb=', (mass_bary/mass_dm)/(0.022068/0.12029))
    

In [29]:
print_bary_dm_frac(sn_MW, R200_MW)

bary/dm= 0.19539545578979162
planck= 0.1834566464377754
fb= 1.065077006387259


In [30]:
print_bary_dm_frac(sn_GSE, R200_GSE)

bary/dm= 0.05050205165641609
planck= 0.1834566464377754
fb= 0.2752805779295944


In [34]:
np.concatenate((sn_MW.part0.GFM_Metals, sn_GSE.part0.GFM_Metals)).shape

(785575, 10)

In [35]:
97.5 + 129

226.5

In [36]:
def separate(pos, vel, Rstart, Vvir, e, pro):
    vrad = np.sqrt(Vvir*Vvir - Vvir*e*Vvir*e)
    vphi = Vvir * e / np.sqrt(2.)

    pos[:,0] += Rstart

    vel[:,0] += -vrad
    vel[:,1] += pro * vphi
    vel[:,2] += vphi

    return pos, vel

def rotate(pos, vel, angle):
    theta = angle * np.pi/180.
    phi = 0.

    pos_rot = np.copy(pos)
    vel_rot = np.copy(vel)

    pos_rot[:,0] = np.cos(phi)*(np.cos(theta)*pos[:,0]+np.sin(theta)*pos[:,1])-np.sin(phi)*pos[:,1]
    pos_rot[:,1] = np.sin(phi)*(np.cos(theta)*pos[:,0]+np.sin(theta)*pos[:,2])+np.cos(phi)*pos[:,1]
    pos_rot[:,2] = -np.sin(theta)*pos[:,0]+np.cos(theta)*pos[:,2]

    vel_rot[:,0] = np.cos(phi)*(np.cos(theta)*vel[:,0]+np.sin(theta)*vel[:,1])-np.sin(phi)*vel[:,1]
    vel_rot[:,1] = np.sin(phi)*(np.cos(theta)*vel[:,0]+np.sin(theta)*vel[:,2])+np.cos(phi)*vel[:,1]
    vel_rot[:,2] = -np.sin(theta)*vel[:,0]+np.cos(theta)*vel[:,2]

    return pos_rot, vel_rot


In [37]:
Rstart = 129.
Vvir = 129.
e = 0.5
pro = 1.
angle = -165

In [45]:
pos, vel = separate(np.array([[0.,0.,0.]]), np.array([[0., 0., 0.]]), Rstart, Vvir, e, pro)
pos, vel = rotate(pos, vel, angle)

In [51]:
def rt_eq1(sn_MW, sn_GSE, R_GSE=129.):
    M_GSE = 0.0
    M_MW = 0.0
    
    for i in range(6):
        M_MW += sn_MW.MassTable[i] * sn_MW.NumPart_Total[i]
        M_GSE += sn_GSE.MassTable[i] * sn_GSE.NumPart_Total[i]
    
    rt = R_GSE * (M_GSE / (2*M_MW))**(1./3.)
    print(rt)
    
    return rt

In [52]:
rt_eq1(sn_MW, sn_GSE)

80.55584819878273


80.55584819878273

In [54]:
def compute_rho0(M_HALOGAS, RC, R200):
    rho0 = M_HALOGAS / ( 4 * np.pi * RC * RC * RC * (10*R200/RC - np.arctan(10*R200/RC) ) )
    return rho0

In [58]:
M_HALOGAS = 0.25 * 49.9148
rho0_MW = compute_rho0(M_HALOGAS, 9, 129)
print(rho0_MW)

M_HALOGAS = 0.18 * 21.5513
rho0_GSE = compute_rho0(M_HALOGAS, 6.5, 97.5)
print(rho0_GSE)

9.608356363277632e-06
7.57281209259249e-06


In [60]:
def compute_rho(r, rho0, rc):
    ans = rho0 * (1 + (r/rc)**2)**(-1)
    return ans

In [62]:
compute_rho(129, rho0_MW, 9)

4.654209217949337e-08

In [68]:
compute_rho(80, rho0_GSE, 6.5)

4.966452883884244e-08

In [72]:
6.5*10

65.0